## Have AI create a new table that contains comments for the recipes as if they were shared on a social media

In [ ]:
# The recipes from bird contained a intro field and those from airtable contained a note field, for the sake of consistency for AI prompts and the database, we will store these as one field named note
%%bigquery
update magazine_recipes_stg.Recipes
set note = intro
where note is null and intro is not null;

Query is running:   0%|          |

""


In [ ]:
# remove the intro field since the information is now stored as note
%%bigquery
create or replace table magazine_recipes_stg.Recipes as
select * except (intro)
from magazine_recipes_stg.Recipes;

Query is running:   0%|          |

""


In [1]:
# Ask the AI to write comments for a recipe given its title, directions, and note
%%bigquery
declare prompt_query STRING default "You are a user of a recipe sharing social media, write a comment for the recipe. Return output with recipe_id, title, and comment.";
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("recipe_id", recipe_id, "title", title, "directions", directions, "note", note))) as prompt
    from magazine_recipes_stg.Recipes as r
    where directions is not null
    order by r.recipe_id
    limit 10
  ),
  struct(TRUE as flatten_json_output)
);

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,## Sweet Potato Breakfast Burritos\n\nThese lo...,"[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
1,"{""recipe_id"":3,""title"":""Spicy Black Bean Nacho...","[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
2,This recipe looks amazing! I love the combinat...,"[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
3,"{""recipe_id"":6,""title"":""Pinto Beans and Tomati...","[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
4,"{""recipe_id"":7,""title"":""Turkey Soup with Homem...","[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
5,## Ham & Potato Soup - Recipe ID: 8\n\nThis so...,"[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
6,## Spanish Chickpeas - A Delicious and Easy Re...,"[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
7,This recipe looks delicious! I can't wait to t...,"[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
8,This looks like a delicious and healthy salad!...,"[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
9,## Lemon Garlic Asparagus with Orzo: A Springt...,"[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...


In [ ]:
# Begin prompt engineering- specifically mention json output
%%bigquery
declare prompt_query STRING default "You are a user of a recipe sharing social media, write a comment for the recipe. Return output as a json with recipe_id, title, and comment.";
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("recipe_id", recipe_id, "title", title, "directions", directions, "note", note))) as prompt
    from magazine_recipes_stg.Recipes as r
    where directions is not null
    order by r.recipe_id
    limit 10
  ),
  struct(TRUE as flatten_json_output)
);

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,"```json\n{""recipe_id"":2,""title"":""Sweet Potato ...",None,,You are a user of a recipe sharing social medi...
1,"```json\n{""recipe_id"":3,""title"":""Spicy Black B...",None,,You are a user of a recipe sharing social medi...
2,"```json\n{""recipe_id"":5,""title"":""Chocolate Ras...",None,,You are a user of a recipe sharing social medi...
3,"```json\n{""recipe_id"":6,""title"":""Pinto Beans a...",None,,You are a user of a recipe sharing social medi...
4,"```json\n{""recipe_id"":7,""title"":""Turkey Soup w...",None,,You are a user of a recipe sharing social medi...
5,"```json\n{""recipe_id"":8,""title"":""Ham & Potato ...",None,,You are a user of a recipe sharing social medi...
6,"```json\n{""recipe_id"":9,""title"":""Spanish Chick...",None,,You are a user of a recipe sharing social medi...
7,"```json\n{""recipe_id"":10,""title"":""Thai Veggie ...",None,,You are a user of a recipe sharing social medi...
8,"```json\n{""recipe_id"": 11, ""title"": ""Quinoa Br...",None,,You are a user of a recipe sharing social medi...
9,"```json\n{""recipe_id"":12,""title"":""Lemon Garlic...",None,,You are a user of a recipe sharing social medi...


In [ ]:
# We are happy with the prompt, but now we want multiple comments for each recipe
%%bigquery
declare prompt_query STRING default "You are a user of a recipe sharing social media, write 3 comments for the recipe (as if you are a different user for each comment). Return output as a json with recipe_id, title, comment1, comment2, comment3.";
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("recipe_id", recipe_id, "title", title, "directions", directions, "note", note, 'ease_of_prep', ease_of_prep))) as prompt
    from magazine_recipes_stg.Recipes as r
    where directions is not null
    order by r.recipe_id
    limit 10
  ),
  struct(TRUE as flatten_json_output)
);

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,"```json\n{""recipe_id"":2,""title"":""Sweet Potato ...",None,,You are a user of a recipe sharing social medi...
1,"```json\n{""recipe_id"":3,""title"":""Spicy Black B...",None,,You are a user of a recipe sharing social medi...
2,"```json\n{""recipe_id"":5,""title"":""Chocolate Ras...",None,,You are a user of a recipe sharing social medi...
3,"```json\n{""recipe_id"":6,""title"":""Pinto Beans a...",None,,You are a user of a recipe sharing social medi...
4,"```json\n{""recipe_id"":7,""title"":""Turkey Soup w...",None,,You are a user of a recipe sharing social medi...
5,"```json\n{""recipe_id"":8,""title"":""Ham & Potato ...",None,,You are a user of a recipe sharing social medi...
6,"```json\n{""recipe_id"":9,""title"":""Spanish Chick...",None,,You are a user of a recipe sharing social medi...
7,"```json\n{""recipe_id"":10,""title"":""Thai Veggie ...",None,,You are a user of a recipe sharing social medi...
8,"```json\n{""recipe_id"":11,""title"":""Quinoa Bruss...",None,,You are a user of a recipe sharing social medi...
9,"```json\n{""recipe_id"":12,""title"":""Lemon Garlic...",None,,You are a user of a recipe sharing social medi...


In [2]:
# all comments are positive sp ask for different levels of satisfaction, the 3 comments result in longer outputs that don't completely close the JSON so we will specify a world limit
%%bigquery
declare prompt_query STRING default "You are a user of a recipe sharing social media, write 3 comments for the recipe (as if you are a different user for each comment, so use a different tone and level of satisfaction). Each comment should be less than 30 words. Return output as a complete json with recipe_id, title, comment1, comment2, comment3.";
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("recipe_id", recipe_id, "title", title, "directions", directions, "note", note, 'ease_of_prep', ease_of_prep))) as prompt
    from magazine_recipes_stg.Recipes as r
    where directions is not null
    order by r.recipe_id
    limit 10
  ),
  struct(TRUE as flatten_json_output)
);

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,"```json\n{""directions"":""Cook sweet potatoes un...","[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
1,"```json\n{""recipe_id"":3,""title"":""Spicy Black B...","[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
2,"```json\n{""recipe_id"":5,""title"":""Chocolate Ras...","[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
3,"```json\n{""recipe_id"":6,""title"":""Pinto Beans a...","[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
4,"```json\n{""recipe_id"":7,""title"":""Turkey Soup w...","[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
5,"```json\n{""recipe_id"":8,""title"":""Ham & Potato ...","[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
6,"```json\n{""recipe_id"":9,""title"":""Spanish Chick...","[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
7,"```json\n{""recipe_id"":10,""title"":""Thai Veggie ...","[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
8,"```json\n{""recipe_id"":11,""title"":""Quinoa Bruss...","[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
9,"```json\n{""recipe_id"":12,""title"":""Lemon Garlic...","[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...


In [5]:
# the AI reuses many comments across recipes, so we will ask for more creativity and see if its any better
%%bigquery
declare prompt_query STRING default "You are a user of a recipe sharing social media, write 3 original and creative comments for the recipe in 25 or fewer words (you are a different user for each comment, so use a different tone and level of satisfaction). Return output as a complete json with recipe_id, title, comment1, comment2, comment3.";
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("recipe_id", recipe_id, "title", title, "directions", directions, "note", note, 'ease_of_prep', ease_of_prep))) as prompt
    from magazine_recipes_stg.Recipes as r
    where directions is not null
    order by r.recipe_id
    limit 5
  ),
  struct(TRUE as flatten_json_output)
);

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,"```json\n{""recipe_id"":2,""title"":""Sweet Potato ...","[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
1,"```json\n{""recipe_id"":3,""title"":""Spicy Black B...","[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
2,"```json\n{""recipe_id"":5,""title"":""Chocolate Ras...","[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
3,"```json\n{""recipe_id"":6,""title"":""Pinto Beans a...","[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...
4,"```json\n{""recipe_id"":7,""title"":""Turkey Soup w...","[{""category"":1,""probability"":1,""probability_sc...",,You are a user of a recipe sharing social medi...


In [ ]:
# we are happy with the output, so we will store it as a table
%%bigquery
declare prompt_query STRING default "You are a user of a recipe sharing social media, write 3 original, creative, and relevant comments for the recipe in 25 or fewer words (you are a different user for each comment, so exhibit a different level of satisfaction). Return output as a CLOSED, COMPLETE json with only recipe_id, comment1, comment2, comment3 ENDING IN }";
create or replace table magazine_recipes_stg_ai.comments as
select *
from ML.generate_text(
  model remote_models.gemini_pro,
  (
    select concat(prompt_query, to_json_string(json_object("recipe_id", recipe_id, "title", title, "directions", directions, "note", note, 'ease_of_prep', ease_of_prep))) as prompt
    from magazine_recipes_stg.Recipes as r
    order by r.recipe_id
  ),
  struct(TRUE as flatten_json_output)
);

Query is running:   0%|          |

""


In [6]:
# explore the comments that were saved
%%bigquery
select * from magazine_recipes_stg_ai.comments
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt
0,"```json\n{""recipe_id"":77,""title"":""Lentil Salsa...",None,,You are a user of a recipe sharing social medi...
1,"```json\n{""recipe_id"":219,""comment1"":""These ta...",None,,You are a user of a recipe sharing social medi...
2,"```json\n{""recipe_id"":239,""comment1"":""This was...",None,,You are a user of a recipe sharing social medi...
3,"```json\n{""recipe_id"":247,""title"":""Whole Wheat...",None,,You are a user of a recipe sharing social medi...
4,"```json\n{""recipe_id"":274,""comment1"":""These or...",None,,You are a user of a recipe sharing social medi...


In [ ]:
# format the json output
%%bigquery
create or replace table magazine_recipes_stg_ai.comments_formatted as
  select trim(replace(replace(replace(replace(replace(ml_generate_text_llm_result, '```json', ''), '```', ''), '\n', ''), ']', ''), '[' , '')) as formatted_comment
  from magazine_recipes_stg_ai.comments

Query is running:   0%|          |

""


In [7]:
%%bigquery
select * from magazine_recipes_stg_ai.comments_formatted
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,formatted_comment
0,"{""directions"":null,""ease_of_prep"":null,""note"":..."
1,"{""recipe_id"":912,""title"":""-Thousand Island Sal..."
2,"{""recipe_id"":916,""comment1"":""This steak sauce ..."
3,"{""recipe_id"":115,""comment1"":""This lasagna was ..."
4,"{""recipe_id"":39,""title"":""Lentil Soup"",""comment..."
5,"{""directions"":""Heat milk in a saucepan over me..."
6,"{""recipe_id"":1059,""comment1"":""This salad is so..."
7,"{""recipe_id"":708,""title"":""Black Bean and Rice ..."
8,"{""recipe_id"":613,""comment1"":""This sauce was am..."
9,"{""recipe_id"":1188,""comment1"":""This was a great..."


In [ ]:
# some comments are still not closed, so we must manually close the json to be able to extract the information
%%bigquery
update magazine_recipes_stg_ai.comments_formatted
set formatted_comment = concat(formatted_comment, '"}')
where formatted_comment not like '%"}%'

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from magazine_recipes_stg_ai.comments_formatted
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,formatted_comment
0,"{""directions"":null,""ease_of_prep"":null,""note"":..."
1,"{""recipe_id"":912,""title"":""-Thousand Island Sal..."
2,"{""recipe_id"":916,""comment1"":""This steak sauce ..."
3,"{""recipe_id"":115,""comment1"":""This lasagna was ..."
4,"{""recipe_id"":39,""title"":""Lentil Soup"",""comment..."
5,"{""directions"":""Heat milk in a saucepan over me..."
6,"{""recipe_id"":1059,""comment1"":""This salad is so..."
7,"{""recipe_id"":708,""title"":""Black Bean and Rice ..."
8,"{""recipe_id"":613,""comment1"":""This sauce was am..."
9,"{""recipe_id"":1188,""comment1"":""This was a great..."


In [ ]:
# create a table that has a field for each comment and a row for each recipe
%%bigquery
CREATE OR REPLACE TABLE magazine_recipes_stg_ai.recipe_comments AS
select
cast(json_value(formatted_comment, '$.recipe_id') as int64) as recipe_id,
json_value(formatted_comment, '$.title') as title,
json_value(formatted_comment, '$.comment1') as comment1,
json_value(formatted_comment, '$.comment2') as comment2,
json_value(formatted_comment, '$.comment3') as comment3
from magazine_recipes_stg_ai.comments_formatted

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from magazine_recipes_stg_ai.recipe_comments
order by recipe_id
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,recipe_id,title,comment1,comment2,comment3
0,<NA>,None,None,None,None
1,<NA>,None,None,None,None
2,1,Chive Butter Radishes,These radishes were so refreshing and flavorfu...,I thought the radishes were a bit too spicy fo...,"I wasn't sure what to expect from this recipe,..."
3,2,None,These were amazing! So easy and perfect for a ...,"I wasn't a huge fan of the sweet potatoes, but...","This is a great base recipe, but I added some ..."
4,3,None,These nachos were amazing! The perfect amount ...,I wasn't a huge fan of these nachos. The beans...,"These nachos were just okay. They weren't bad,..."
5,4,Balsamic Pork Chops,These pork chops were amazing! So juicy and fl...,I thought this recipe was just okay. The pork ...,This recipe was a disaster! The pork chops wer...
6,5,Chocolate Raspberry Torte,This looks amazing! Can't wait to try it.,I made this last night and it was a huge hit! ...,This recipe was a bit more difficult than I ex...
7,6,Pinto Beans and Tomatillo Cilantro Lime Rice,This was so good! I added some extra cilantro ...,"I'm not a huge fan of beans, but this was actu...","This was a bit too spicy for me, but I think i..."
8,7,None,This soup was so easy to make and tasted amazi...,"I wasn't sure about this recipe, but I'm so gl...","This soup was just okay. It wasn't bad, but it..."
9,8,None,This soup was amazing! So easy to make and ful...,I wasn't a huge fan of this soup. It was a bit...,"This soup was just okay. It wasn't bad, but it..."


In [ ]:
# create a table that stores each comment as its own row
%%bigquery
create or replace table magazine_recipes_stg_ai.all_comments as
select recipe_id, comment1 as comment
from magazine_recipes_stg_ai.recipe_comments
union all
select recipe_id, comment2 as comment
from magazine_recipes_stg_ai.recipe_comments
union all
select recipe_id, comment3 as comment
from magazine_recipes_stg_ai.recipe_comments

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from magazine_recipes_stg_ai.all_comments
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,recipe_id,comment
0,256,"Meh, not a fan. The texture was a bit weird an..."
1,256,"Good, but a bit too sweet for me. I might try ..."
2,256,This dipping sauce is amazing! So easy to make...
3,512,This recipe was a bit too sweet for me. I thin...
4,512,This was a great way to use up leftover cherry...
5,512,"I wasn't sure about this recipe, but I'm glad ..."
6,768,"I'm not a huge fan of melon, but this was actu..."
7,768,"I wasn't sure what to expect, but this was sur..."
8,768,This melon is so refreshing and perfect for a ...
9,1024,This was a great recipe! The turkey was so moi...


In [ ]:
# create a final staging table that contain all comments for the recipes, assign a random user_id for the person who let the comment
%%bigquery
create or replace table magazine_recipes_stg.Comments as
SELECT ROW_NUMBER() OVER () AS comment_id,
      p.post_id,
      cast(FLOOR(RAND() * (select max(user_id) from magazine_recipes_stg.Users) + 1) as int64) AS user_id,
      c.comment,
      'ai' as data_source,
      current_timestamp() as load_time
from magazine_recipes_stg_ai.all_comments c
left join magazine_recipes_stg.Posts p on c.recipe_id = p.recipe_id
where comment is not null

Query is running:   0%|          |

""


In [ ]:
%%bigquery
select * from magazine_recipes_stg.Comments
limit 10

Query is running:   0%|          |

Downloading:   0%|          |

,comment_id,post_id,user_id,comment,data_source,load_time
0,38,416,93,This is the perfect summer treat! So refreshin...,ai,2024-04-20 04:08:04.629564+00:00
1,61,760,27,This recipe was easy to follow and turned out ...,ai,2024-04-20 04:08:04.629564+00:00
2,194,1080,90,This was absolutely delicious! The Spa Creme w...,ai,2024-04-20 04:08:04.629564+00:00
3,230,255,17,"I'm not a huge fan of raisins, but these were ...",ai,2024-04-20 04:08:04.629564+00:00
4,407,245,67,"Meh, not impressed. The texture was a bit off. 😕",ai,2024-04-20 04:08:04.629564+00:00
5,532,914,79,I thought these cookies were a bit dry. I migh...,ai,2024-04-20 04:08:04.629564+00:00
6,557,508,20,I thought this recipe was just okay. The sauce...,ai,2024-04-20 04:08:04.629564+00:00
7,585,1140,42,These kebabs were amazing! The dressing was th...,ai,2024-04-20 04:08:04.629564+00:00
8,588,1088,29,"I wasn't sure about this recipe, but I'm so gl...",ai,2024-04-20 04:08:04.629564+00:00
9,677,1131,27,This looks delicious! I can't wait to try it.,ai,2024-04-20 04:08:04.629564+00:00


In [ ]:
# check if any posts have two comments by the same user (they might have different sentiments so we do not want that)
%%bigquery
select count(c1.post_id)
from magazine_recipes_stg.Comments c1
join magazine_recipes_stg.Comments c2 on c1.post_id = c2.post_id and c1.user_id = c2.user_id and c1.comment_id != c2.comment_id


Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,40


In [ ]:
# if any posts have comments by the same user and change the user
%%bigquery
update magazine_recipes_stg.Comments
set user_id =
    CASE
    WHEN MOD((user_id + length(comment)), (select max(user_id) from magazine_recipes_stg.Users)) != 0 THEN MOD((user_id + length(comment)), (select max(user_id) from magazine_recipes_stg.Users))
    ELSE MOD((user_id + length(comment)), (select max(user_id) from magazine_recipes_stg.Users)) + 1
    END
where post_id in (
select c1.post_id from magazine_recipes_stg.Comments c1
join magazine_recipes_stg.Comments c2 on c1.post_id = c2.post_id and c1.user_id = c2.user_id and c1.comment_id != c2.comment_id)

Query is running:   0%|          |

""


In [ ]:
# check if any posts still have two comments by the same user
%%bigquery
select count(c1.post_id)
from magazine_recipes_stg.Comments c1
join magazine_recipes_stg.Comments c2 on c1.post_id = c2.post_id and c1.user_id = c2.user_id and c1.comment_id != c2.comment_id


Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,0


In [9]:
# check if any of the commentors are the same as the poster and make it a different commentor

%%bigquery
update magazine_recipes_stg.Comments
SET user_id =
    CASE
        WHEN user_id < (SELECT MAX(user_id) FROM magazine_recipes_stg.Users) THEN user_id + 1
        ELSE user_id - 1
    END
WHERE post_id IN (
    SELECT p.post_id
    FROM magazine_recipes_stg.Comments c
    JOIN magazine_recipes_stg.Posts p ON c.user_id = p.user_id AND c.post_id = p.post_id
);

Query is running:   0%|          |

""


In [10]:
# see if any still meet this criteria
%%bigquery
SELECT count(p.post_id)
    FROM magazine_recipes_stg.Comments c
    JOIN magazine_recipes_stg.Posts p ON c.user_id = p.user_id AND c.post_id = p.post_id

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,0


In [ ]:
# determine the possible range of time to use in randomly assigned timestamps (must be less than the amount of time from latest post to now)
%%bigquery
select TIMESTAMP_DIFF(CURRENT_TIMESTAMP(), (select max(timestamp_posted) from magazine_recipes_stg.Posts), SECOND)

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,6108032


In [ ]:
# add a randomly assigned timestamp for the comment within the appropriate range
%%bigquery
alter table magazine_recipes_stg.Comments
add column timestamp_commented TIMESTAMP;

UPDATE magazine_recipes_stg.Comments c
SET timestamp_commented = TIMESTAMP_ADD(
    (SELECT p.timestamp_posted FROM magazine_recipes_stg.Posts p WHERE p.post_id = c.post_id),
    INTERVAL CAST(FLOOR(RAND() * 6108032) AS INT64) SECOND)
where 1 = 1;


Query is running:   0%|          |

""


Add primary and forgein keys and check that the constraints are satisfied

In [ ]:
# add keys to this table
%%bigquery
alter table magazine_recipes_stg.Comments
add primary key (comment_id) not enforced;

alter table magazine_recipes_stg.Comments
add foreign key (post_id) references magazine_recipes_stg.Posts(post_id) not enforced;

alter table magazine_recipes_stg.Comments
add foreign key (user_id) references magazine_recipes_stg.Users(user_id) not enforced;

Query is running:   0%|          |

""


In [ ]:
# check key constraints
%%bigquery
select * from magazine_recipes_stg.Comments
where post_id not in (select post_id from magazine_recipes_stg.Posts)

Query is running:   0%|          |

Downloading: |          |

,comment_id,post_id,user_id,comment,data_source,load_time,timestamp_commented


In [ ]:
# check key constraints
%%bigquery
select * from magazine_recipes_stg.Comments
where user_id not in (select user_id from magazine_recipes_stg.Users)

Query is running:   0%|          |

Downloading: |          |

,comment_id,post_id,user_id,comment,data_source,load_time,timestamp_commented
